In [1]:
pip install fuzzywuzzy

In [115]:
import numpy as np
import pandas as pd

df_csv = pd.read_csv("G:/Krowdy/instituciones_educativas.csv")
df_json = pd.read_json("G:/Krowdy/Universidades.json")

In [116]:
df_csv.shape

(8588, 2)

In [117]:
## Parte 1

#para visualizar
#df_csv[df_csv["value"].str.contains("-")]

df_csv['campo3'] = df_csv.apply(lambda row: row['value'].split('-')[1] if '-' in row['value'] else row['value'], axis=1)

df_csv['campo3'] = df_csv['campo3'].str.replace(r'\s+', '', regex=True)

df_csv["campo3"] = df_csv["campo3"].str.lower()
df_json["Siglas2"] = df_json["Siglas "].str.lower()

df_parte1 = pd.merge(df_csv,df_json, left_on="campo3", right_on="Siglas2", how="left")


df_parte1['tipo'] = np.where(df_parte1['Nombre '].isna(), 'Nombre', 'Sigla')

drop_columns=["campo3","código INEI", "Siglas ", "Siglas2"]

df_parte1.drop(columns=drop_columns, inplace = True)

df_parte1.head()



,candidateId,value,Nombre,tipo
0,630633fa1e599d0009d6dee0,Ucv,Universidad César Vallejo S.A.C.,Sigla
1,630633fa1e599d0009d6dee0,José Gálvez,NaN,Nombre
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Universidad Peruana de Ciencias Aplicadas S.A.C.,Sigla
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,NaN,Nombre
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,NaN,Nombre


In [118]:
## Parte 2

df_parte2 = df_parte1[(df_parte1["tipo"] == "Nombre")]

df_parte2["value2"] = df_parte2["value"].str.lower()

df_parte2 = df_parte2[~df_parte2["value2"].str.contains("instituto")]
df_parte2 = df_parte2[~df_parte2["value2"].str.contains("colegio")]
df_parte2 = df_parte2[~df_parte2["value2"].str.contains("universidad")]

df_parte2["value"] = df_parte2["value"].apply(lambda x: "Universidad "+x)

df_parte2 = pd.merge(df_parte1,df_parte2, left_index=True, right_index=True, how="left")

drop_columns=["candidateId_y","Nombre _y", "tipo_y", "value2"]

df_parte2.drop(columns=drop_columns, inplace = True)

# Definir la función lambda que evalúa los valores de campo1 y campo2
def evaluar_campos(row):
    if pd.isnull(row['value_y']):
        return row['value_x']
    else:
        return row['value_y']

# Aplicar la función a cada fila del DataFrame utilizando el método apply()
df_parte2['value_y'] = df_parte2.apply(lambda row: evaluar_campos(row), axis=1)

df_parte2["value_y"] = df_parte2["value_y"].str.lower()
df_json["Nombre "] = df_json["Nombre "].str.lower()

df_parte2.head(10)

C:\Users\Ricardo\AppData\Local\Temp\ipykernel_9832\93286676.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parte2["value2"] = df_parte2["value"].str.lower()


,candidateId_x,value_x,Nombre _x,tipo_x,value_y
0,630633fa1e599d0009d6dee0,Ucv,Universidad César Vallejo S.A.C.,Sigla,ucv
1,630633fa1e599d0009d6dee0,José Gálvez,NaN,Nombre,universidad josé gálvez
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Universidad Peruana de Ciencias Aplicadas S.A.C.,Sigla,universidad peruana de ciencias aplicadas - upc
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,NaN,Nombre,universidad centro peruano de estudios bancar...
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,NaN,Nombre,instituto de formación bancaria - ifb
5,63064ec3d7fa700008e7ed19,Certus,NaN,Nombre,universidad certus
6,63064fc2e86a980008de8c83,Cepeban,NaN,Nombre,universidad cepeban
7,630653281e599d0009d6e817,Ifb Certus,NaN,Nombre,universidad ifb certus
8,63065b8529da5b00089fbb2f,Instituto Superior Sise,NaN,Nombre,instituto superior sise
9,63065d9d0b25100008421575,Argentina,NaN,Nombre,universidad argentina


In [119]:
from fuzzywuzzy import fuzz, process


# Función para homologar universidades
def homologar_universidad(value_y):
    match = process.extractOne(value_y, df_json['Nombre '], scorer=fuzz.token_sort_ratio)
    if match[1] >= 90:
        return match[0]
    else:
        return ''

# Crear columna de universidad homologada en df_parte2
df_parte2['Universidad_Homologada'] = df_parte2['value_y'].apply(homologar_universidad)
df_parte2.head(20)

,candidateId_x,value_x,Nombre _x,tipo_x,value_y,Universidad_Homologada
0,630633fa1e599d0009d6dee0,Ucv,Universidad César Vallejo S.A.C.,Sigla,ucv,
1,630633fa1e599d0009d6dee0,José Gálvez,NaN,Nombre,universidad josé gálvez,
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Universidad Peruana de Ciencias Aplicadas S.A.C.,Sigla,universidad peruana de ciencias aplicadas - upc,
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,NaN,Nombre,universidad centro peruano de estudios bancar...,
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,NaN,Nombre,instituto de formación bancaria - ifb,
5,63064ec3d7fa700008e7ed19,Certus,NaN,Nombre,universidad certus,
6,63064fc2e86a980008de8c83,Cepeban,NaN,Nombre,universidad cepeban,
7,630653281e599d0009d6e817,Ifb Certus,NaN,Nombre,universidad ifb certus,
8,63065b8529da5b00089fbb2f,Instituto Superior Sise,NaN,Nombre,instituto superior sise,
9,63065d9d0b25100008421575,Argentina,NaN,Nombre,universidad argentina,


In [46]:
##CASO 1 - 1090 UNIVERSIDADES HOMOLOGADAS

def homologar_universidad(value):
    for index, row in df_json.iterrows():
        if row['Nombre '] in value:
            return row['Nombre ']
    return ''

df_parte2['Universidad_Homologada'] = df_parte2['value_y'].apply(homologar_universidad)

df_parte2.head()

,candidateId_x,value_x,Nombre _x,tipo_x,value_y,Universidad_Homologada
0,630633fa1e599d0009d6dee0,Ucv,Universidad César Vallejo S.A.C.,Sigla,ucv,
1,630633fa1e599d0009d6dee0,José Gálvez,NaN,Nombre,universidad josé gálvez,
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Universidad Peruana de Ciencias Aplicadas S.A.C.,Sigla,universidad peruana de ciencias aplicadas - upc,
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,NaN,Nombre,universidad centro peruano de estudios bancar...,
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,NaN,Nombre,instituto de formación bancaria - ifb,


In [120]:
# Definir la función lambda que evalúa los valores de campo1 y campo2
def evaluar_campos(row):
    if row['Universidad_Homologada'] == "":
        return row['Nombre _x']
    else:
        return row['Universidad_Homologada']

# Aplicar la función a cada fila del DataFrame utilizando el método apply()
df_parte2['Universidad_Homologada'] = df_parte2.apply(lambda row: evaluar_campos(row), axis=1)

drop_columns=["Nombre _x","value_y"]

df_parte2.drop(columns=drop_columns, inplace = True)

df_parte2["Universidad_Homologada"] = df_parte2["Universidad_Homologada"].fillna("No Encontrado")

df_parte2.head()

,candidateId_x,value_x,tipo_x,Universidad_Homologada
0,630633fa1e599d0009d6dee0,Ucv,Sigla,Universidad César Vallejo S.A.C.
1,630633fa1e599d0009d6dee0,José Gálvez,Nombre,No Encontrado
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Sigla,Universidad Peruana de Ciencias Aplicadas S.A.C.
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,Nombre,No Encontrado
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,Nombre,No Encontrado


In [121]:
df_parte2[(df_parte2["Universidad_Homologada"] != "No Encontrado")].shape

(2202, 4)

In [114]:
df_parte2.to_csv('G:/Krowdy/salida_4.csv', index=False)

In [123]:
df_parte2[(df_parte2["Universidad_Homologada"] != "")&(~df_parte2["value_x"].str.contains("Universidad"))]


,candidateId_x,value_x,tipo_x,Universidad_Homologada
0,630633fa1e599d0009d6dee0,Ucv,Sigla,Universidad César Vallejo S.A.C.
1,630633fa1e599d0009d6dee0,José Gálvez,Nombre,No Encontrado
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,Nombre,No Encontrado
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,Nombre,No Encontrado
5,63064ec3d7fa700008e7ed19,Certus,Nombre,No Encontrado
...,...,...,...,...
8580,634f51c11528ff0009d07c0c,Zegel Ipae,Nombre,No Encontrado
8582,634f5f73bf94c20009cf469f,Colegio Particular Divina Misericordia,Nombre,No Encontrado
8583,634f5f73bf94c20009cf469f,Colegio Particular Nuestra Señora De Monserrat,Nombre,No Encontrado
8586,6312a4a8e5ee99003f0863ac,Instituto De Formación Bancaria - Ifb,Nombre,No Encontrado
